<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Financial Fraud Detection using Traditional approach 
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Traditionl Approach</b></p>
<p style = 'font-size:16px;font-family:Arial'>
    ClearScape Analytics provides powerful, flexible end-to-end data connectivity, feature engineering, model training, evaluation, and operational functions that can be deployed at scale as enterprise data assets; treating the products of ML and AI as first-class analytic processes in the enterprise. With ClearScape Analytics, data scientists can use their preferred language, tools and platform to develop models to identify this fraud. Even in large scale operations, users have the guarantee that Vantage can scale to their needs and reduce fraud.</p>
    
<p style = 'font-size:20px;font-family:Arial'><b>Overview</b></p>
<p style = 'font-size:16px;font-family:Arial'>
    This notebook demonstrates an end-to-end Financial Fraud Detection workflow using Teradata ClearScape Analytics,
enhanced with the Enterprise Feature Store (EFS) lineage management for <b>Week 1</b> and <b>Week 2</b> datasets.
It showcases feature engineering, model training, scoring, and evaluation using Vantage In-DB capabilities.</p>

<p style = 'font-size:16px;font-family:Arial'>Below are the steps involved in traditional approach:</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Prepare data: </b>ClearScape Analytics offers highly optimized in-database functions for data preparation, minimizing data movement and enabling the enterprise feature store.</li>
    <li><b>Train models: </b>ClearScape Analytics provides vertical and horizontal scaling capabilities that make it possible to efficiently train any number of models — from a few to a few million.</li>
    <li><b>Deploy models: </b>ClearScape Analytics integrates model scoring with business data, both in real time and batch scoring, for effective operationalization and automated monitoring of AI models.</li>
    
</ul>

<p style = 'font-size:18px;font-family:Arial'>In the traditional approach mentioned, we will follow the below steps:</p>    
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Data Collection</li>
    <li>Data Exploration</li>
    <li>Enterprise feature store</li>
    <li>Data Preparation using widgets</li>
    <li>Model Training(2-3 different models)</li>
    <li>Model Evaluation using ROC and Confusion Matrix</li>
    <li>Best performing model</li>
    <li>Model Scoring using best model</li>
    <li>Operationalize Model using ModelOps</li>
</ul>

<p style = 'font-size:18px;font-family:Arial'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial'>To maximize the business value of advanced analytic techniques including Machine Learning and Artificial Intelligence, it is estimated that organizations must scale their model development and deployment pipelines to 100s or 1000s of times greater amounts of data, models, or both.</p>    

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Configuring the Environment</b>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# %%capture
# !pip install teradataml --upgrade

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>Please execute the above pip install to get the latest version of the required library. Be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
# Standard Libraries
import os
import getpass
import warnings
warnings.filterwarnings("ignore")

# Teradata Libraries
from teradataml import *

# Configuration
spacing_large = " "*95
spacing_small = " "*12
display.max_rows = 5
configure.val_install_location = 'val'

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql("SET query_band='DEMO=EE_week1andweek2.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:16px;font-family:Arial'>We begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_GLM_Fraud_cloud');"        # Takes 1 minute
%run -i ../../run_procedure.py "call get_data('DEMO_GLM_Fraud_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>We loaded the data from <a href = 'https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data'>https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data</a> into Vantage in a table named "transaction_data". We checked the data size and printed sample rows: 63k rows and 12 columns.</p>
<p style = 'font-size:16px;font-family:Arial'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>

In [ ]:
txn_data = DataFrame(in_schema('DEMO_GLM_Fraud', 'transaction_data'))
# txn_data = DataFrame(in_schema('demo_user', 'transaction_data'))
print(txn_data.shape)
txn_data

<p style = 'font-size:16px;font-family:Arial'>In this simulated scenario, deceptive agents engage in transactions with the objective of taking control of customers' accounts, transferring funds to another account, and ultimately cashing out for profit.</p>

<p style = 'font-size:18px;font-family:Arial'><b>3.1 How many fraudulent transactions do we have in our dataset?</b></p>

In [ ]:
# There are 92 fraud transactions i.e. 0.14% of fraud transactions in the dataset.
print("No of fraud transactions: %d\nPercentage of fraud transactions: %.2f%%"%(
    txn_data.loc[txn_data.isFraud == 1].shape[0],
    txn_data.loc[txn_data.isFraud == 1].shape[0]/txn_data.shape[0]*100)
)

<br>
<p style = 'font-size:18px;font-family:Arial'><b>3.2 How many transactions do we have group by transaction type?</b></p>

In [ ]:
# Filter data for fraud transactions and group by 'type'
transactions_by_type = txn_data.groupby('type').count().get(['type','count_txn_id'])


# Sort by 'count_step' column in descending order
transactions_by_type = transactions_by_type.sort('count_txn_id', ascending = False)

transactions_by_type = transactions_by_type.assign(
    type_int = case([
        (transactions_by_type.type == 'CASH_IN', 0),
        (transactions_by_type.type == 'CASH_OUT', 1),
        (transactions_by_type.type == 'DEBIT', 2),
        (transactions_by_type.type == 'PAYMENT ', 3),
        (transactions_by_type.type == 'TRANSFER', 4),
    ])
)

In [ ]:
transactions_by_type.plot(
    x = transactions_by_type.type_int,
    y = transactions_by_type.count_txn_id,
    kind = 'bar',
    legend = ['Count by Type'],
    ylabel = 'Count of Transactions',
    xlabel = spacing_small.join(sorted(list(transactions_by_type[['type']].get_values().flatten()))),
    title = "Number of Transactions per Transaction Type"
)

<br>
<p style = 'font-size:18px;font-family:Arial'><b>3.3 How many fraudulent transactions do we have group by transaction type?</b></p>

In [ ]:
# Filter data for fraud transactions and group by 'type'
fraud_transactions_by_type = txn_data.loc[txn_data.isFraud == 1].groupby('type').count().get(['type','count_txn_id'])

# Sort by 'count_step' column in descending order
fraud_transactions_by_type = fraud_transactions_by_type.sort('count_txn_id', ascending = False)

fraud_transactions_by_type = fraud_transactions_by_type.assign(
    total_fraud = txn_data.loc[txn_data.isFraud == 1].shape[0],
    type_int = case([(fraud_transactions_by_type.type == 'TRANSFER', 0)], else_ = 1)
)

In [ ]:
fraud_transactions_by_type.plot(
    x = fraud_transactions_by_type.type_int,
    y = [fraud_transactions_by_type.total_fraud, fraud_transactions_by_type.count_txn_id],
    kind = 'bar',
    figsize = (800, 500),
    legend = ['Total Fraud', 'Count by Type'],
    ylabel = 'Count of Fraud Transactions',
    xlabel = 'TRANSFER' + spacing_large + 'CASH_OUT',
    title = "Number of Fraud Transactions by Transaction Type"
)

<p style = 'font-size:16px;font-family:Arial'>From the above result, we can see that out of the 92 fraud transactions, 47 are from transaction type "TRANSFER" and 45 are from "CASH_OUT".</p>

<br>
<p style = 'font-size:18px;font-family:Arial'><b>3.4 What percentage of fraudulent transactions do we have where transaction amount is equal to old balance in the origin account?</b></p>

<p style = 'font-size:16px;font-family:Arial'>This might be the case where the fraudster emptied the account of the victim.</p>

In [ ]:
print("No of cleanout fraud transactions: %d\nPercentage of cleanout fraud transactions: %.2f%%"%(
    txn_data.loc[txn_data['amount'] == txn_data.oldbalanceOrig].loc[txn_data['isFraud'] == 1].shape[0],
    txn_data.loc[txn_data['amount'] == txn_data.oldbalanceOrig].loc[txn_data['isFraud'] == 1].shape[0] / txn_data.loc[txn_data.isFraud == 1].shape[0]*100)
)

<p style = 'font-size:16px;font-family:Arial'>From the above result, we can see that out of 92 Fraud transactions, the amount involved in 90 fraud transactions was equal to the total balance in the account. </p>

<p style = 'font-size:16px;font-family:Arial'><b>Below are some insights about the dataset:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>We have 92 fraud transactions, which account for 0.14% of the dataset.</li>
    <li>Out of these 92 fraud transactions, 47 are of type TRANSFER, and 45 are of type CASH_OUT.</li>
    <li>Approximately 97.83% of our fraud transactions have a transaction amount equal to oldbalanceOrig, indicating account cleanout.</li>
    <li>About 71.74% of our fraud transactions have the recipient's old balance as zero.</li>
    <li>The isFlaggedFraud indicator is correct only two times among our 92 fraud transactions.</li>
</ol>

<br>
<p style = 'font-size:18px;font-family:Arial'><b>3.5 Univariate statistics</b></p>

<p style = 'font-size:16px;font-family:Arial'>The describe function computes the count, mean, std, min, percentiles, and max for numeric columns.</p>

In [ ]:
txn_data.describe()

<br>
<p style = 'font-size:18px;font-family:Arial'><b>3.6 Checking for Null Values</b></p>
<p style = 'font-size:16px;font-family:Arial'>The ColumnSummary() function can be used to take a quick look at the columns, their datatypes, and summary of NULLs/non-NULLs for a given table.</p>

In [ ]:
colsum = ColumnSummary(
    data  = txn_data,
    target_columns = [':']
)
colsum.result

<br>
<b style='font-size:18px;font-family:Arial'>3.7 Week-wise Dataset Split</b>
<p style='font-size:16px;font-family:Arial'>
The dataset is divided into <b>Week 1</b> and <b>Week 2</b> for separate feature lineage tracking.
This simulates time-based feature evolution in financial systems.
</p>


In [ ]:
split_result = txn_data.sample(frac=[0.3, 0.7], seed=42, id_column="txn_id")

week1_df = split_result.loc[split_result.sampleid == 1]
week2_df = split_result.loc[split_result.sampleid == 2]

<p style='font-size:16px;font-family:Arial'>
Week 1 and Week 2 datasets represent sequential transactional windows.
Each set is used to derive different sets of engineered features within the EFS.
</p>

In [ ]:
print("Week1:", week1_df.shape)
print("Week2:", week2_df.shape)

<b style = 'font-size:20px;font-family:Arial'>4. Feature Engineering</b>

<p style='font-size:16px;font-family:Arial'>Teradata Enterprise Feature Store (EFS) Functions are designed to handle feature management within the Vantage environment. While inspired by the syntax of Feast, Teradata EFS Functions stands out, offering efficiency and robustness in data management and feature handling tailored specifically for the use of Teradata Vantage. Teradata EFS Functions use Teradata Dataframes for Feature management, to the contrary of the pandas dataframe of Feast. With Teradata Dataframes we avoid extracting the data to create or use Features from the Enterprise Feature Store (EFS). The EFS Functions are crafted to empower Data Science teams for effective and streamlined feature management. This notebook will walk you through the capabilities of EFS Functions, demonstrating how it integrates seamlessly with your data models and processes.</p>

<br>
<b style = 'font-size:18px;font-family:Arial'>4.1 Setup a Feature Store Repository</b>

<p style='font-size:16px;font-family:Arial'>The Enterprise Feature Store (EFS) SDK is designed with a totally object-oriented approach, focusing on intuitive interaction with feature stores. Central to this design are several core objects: Feature, Entity, DataSource, FeatureGroup. Together, these objects facilitate the efficient management and utilization of features within your data ecosystem, leveraging Teradata Vantage for metadata storage.</p>
<p style='font-size:16px;font-family:Arial'>A feature store repository serves as the foundational environment for storing and managing your data features. The owner of the FeatureStore can grant/revoke read only, write only or read and write authorization to other user(s)</p>

<p style='font-size:16px;font-family:Arial'> Create the Feature Store</p>

In [ ]:
fs = FeatureStore(repo="financial_fraud", data_domain="transaction_features")

<p style='font-size:16px;font-family:Arial'> Setup the Feature Store</p>

In [ ]:
fs.setup()

In [ ]:
fs = FeatureStore(repo="financial_fraud", data_domain="transaction_features")

In [ ]:
# List whether FeatureStore is setup or not.
fs.list_repos()

<br>
<p style='font-size:18px;font-family:Arial'><b>4.2 Week 1 Features: Foundational Transaction Behavior</b></p>
<p style='font-size:16px;font-family:Arial'>
Week 1 focuses on the <b>origin account’s behavior</b> — how much money is moving out and how the balances change over time.
These features establish the baseline of a sender’s transaction patterns, helping to detect anomalies in withdrawal and transfer patterns.
</p>

<ul style='font-size:16px;font-family:Arial'>
  <li><b>step</b>: Represents the transaction’s time sequence. It helps detect whether frauds occur at specific time intervals or bursts, which is common in coordinated attacks.</li>
  <li><b>oldbalanceOrig</b>: The balance in the origin (sender) account before the transaction. A sudden or total depletion of this balance is a strong fraud indicator.</li>
  <li><b>newbalanceOrig</b>: The balance in the origin account after the transaction. Comparing old and new balances helps quantify the transaction magnitude and detect “cleanout” patterns (when the entire account is emptied).</li>
</ul>

<p style='font-size:16px;font-family:Arial'>
In summary, Week 1 features define <b>“how the sender behaves.”</b> These features are crucial for early anomaly detection, since many fraudulent actions start with abnormal outgoing transactions or depleted source accounts.
</p>


In [ ]:
# Making Features
week1_df = week1_df.assign(
    
    step = week1_df.step,
    newbalanceOrig = week1_df.newbalanceOrig,
    oldbalanceOrig = week1_df.oldbalanceOrig
)

<p style='font-size:16px;font-family:Arial'> We are storing the transformation here. So, even if underlying data varies, the data transformation steps remain same.</p>

In [ ]:
# Create a view
week1_view = week1_df.create_view("fraud_week1_view")

<p style='font-size:16px;font-family:Arial'>Creating the FeatureProcess </p>

In [ ]:
fp1 = FeatureProcess(
    repo="financial_fraud",
    data_domain="transaction_features",
    object=week1_view,
    entity="txn_id",
    features=[
        "step",
        "newbalanceOrig",
        "oldbalanceOrig"
    ],
    description="Week1 features capturing balance and transaction ratios"
)
fp1.run()

In [ ]:
# List the feature process.
fs.list_feature_processes()

<p style='font-size:16px;font-family:Arial'>We ingested three features <code>step</code>, <code>newbalanceOrig</code>, and <code>oldbalanceOrig </code> from a single feature process. This demonstrates how multiple related features can be managed and tracked together within the feature store, maintaining their lineage to the originating process.</p>

In [ ]:
fs.mind_map()

<br>
<p style='font-size:18px;font-family:Arial'><b>4.3 Week 2 Features: Downstream Transaction Linkage</b></p>

<p style='font-size:16px;font-family:Arial'>
Week 2 expands the feature lineage to include <b>destination and relational features</b>, capturing how funds propagate through the system.
Fraudulent activity often involves multiple accounts where money is transferred and quickly moved again to obscure its source.
</p>

<ul style='font-size:16px;font-family:Arial'>
  <li><b>nameOrig</b>: Encodes the originating account ID. Used for entity mapping and relationship tracking across transactions.</li>
  <li><b>nameDest</b>: The destination account ID. Monitoring repeated destinations helps detect mule accounts or hubs of fraudulent transfers.</li>
  <li><b>oldbalanceDest</b>: The destination account’s balance before the transaction. If it’s zero, it could indicate a newly created or rarely used account used for fraudulent inflow.</li>
  <li><b>newbalanceDest</b>: The destination account’s balance after the transaction. Large sudden increases can indicate funds consolidation or laundering behavior.</li>
</ul>

<p style='font-size:16px;font-family:Arial'>
Together, these features characterize <b>“how the money flows through the network.”</b> This second lineage adds situational awareness, helping the model learn destination-based patterns such as zero-balance inflows, cascading transfers, and dormant account activations - all red flags for financial fraud.
</p>


In [ ]:
week2_df = week2_df.assign(
    
    step = week2_df.step,
    newbalanceOrig = week2_df.newbalanceOrig,
    oldbalanceOrig = week2_df.oldbalanceOrig
)
week2_df = week2_df.assign(
    nameOrig = week2_df.nameOrig,
    nameDest =  week2_df.nameDest,
    oldbalanceDest = week2_df.oldbalanceDest,
    newbalanceDest = week2_df.newbalanceDest
)

week2_df

<p style='font-size:16px;font-family:Arial'> We are storing the transformation here. So, even if underlying data varies, the data transformation steps remain same.</p>

In [ ]:
# Create a view
week2_view = week2_df.create_view("fraud_week2_view")

<p style='font-size:16px;font-family:Arial'>Creating the FeatureProcess </p>

In [ ]:
fp2 = FeatureProcess(
    repo="financial_fraud",
    data_domain="transaction_features",
    object=week2_view,
    entity="txn_id",
    features=[
        "step",
        "newbalanceOrig",
        "oldbalanceOrig",
        "nameOrig",
        "nameDest",
        "oldbalanceDest",
        "newbalanceDest"
    ],
    description="Week2 features detecting zero balance and full account cleanouts"
)
fp2.run()

In [ ]:
# List the feature process.
fs.list_feature_processes()

In [ ]:
fs.mind_map()

<br>
<b style='font-size:18px;font-family:Arial'>4.4 Building a Curated Dataset</b>
<p style='font-size:16px;font-family:Arial'>
Both Week 1 and Week 2 feature processes are merged into a curated dataset
that serves as the foundation for subsequent modeling.
</p>

In [ ]:
dc_customer = fs.get_dataset_catalog()

selected_profile_features = {
        'step': fp2.process_id,
        'newbalanceOrig': fp2.process_id,
        'oldbalanceOrig': fp2.process_id,
        'nameOrig': fp2.process_id,
        'nameDest': fp2.process_id,
        'oldbalanceDest': fp2.process_id,
        'newbalanceDest': fp2.process_id
    }

tdf = dc_customer.build_dataset(entity='txn_id',
                          selected_features=selected_profile_features,
                          view_name="fraud_dataset",
                          description="Curated dataset for financial fraud")

In [ ]:
fs.list_dataset_catalogs()

In [ ]:
fs.mind_map()

<br>
<b style='font-size:18px;font-family:Arial'>4.5 Explore the FeatureStore</b>
<p style='font-size:16px;font-family:Arial'>
The <b>Feature Catalog</b> acts as 
a centralized registry for all feature definitions within a given repository and data domain. 
It allows users to efficiently organize, search, and retrieve reusable features for various analytical and machine learning workflows.</p>

In [ ]:
fc = FeatureCatalog(repo="financial_fraud",
                    data_domain="transaction_features")
fc

<p style='font-size:16px;font-family:Arial'>The <code>list_feature_versions()</code> function provides visibility into this versioning history, enabling precise tracking of feature evolution over time.
</p>

In [ ]:
f_ver = fc.list_feature_versions()
f_ver

<p style='font-size:16px;font-family:Arial'>
After registering and versioning the features within the Enterprise Feature Store (<b>EFS</b>), 
each feature process automatically creates a corresponding physical table in the underlying repository schema. 
These tables follow a standardized naming convention such as 
<code>FS_T_&lt;UUID&gt;</code>, where the unique identifier corresponds to a specific <b>FeatureProcess</b>.
</p>

In [ ]:
feat_df = DataFrame(in_schema('financial_fraud', 'FS_T_0b9c6f83_0f98_78ec_e504_ec0a5017380b'))
feat_df

<p style='font-size:16px;font-family:Arial'>The <code>features</code> property of the feature catalog lists all features currently registered in the catalog.</p>

In [ ]:
fc.features

<p style='font-size:16px;font-family:Arial'>The <code>data_domain</code> property shows the business domain associated with the feature catalog.</p>

In [ ]:
fc.data_domain

<p style='font-size:16px;font-family:Arial'>
The <code>list_features()</code> function provides a detailed view of all features ingested within the current repository, 
including their names, data domains, data types, and the originating <b>FeatureProcess</b> ID. 
This serves as a quick validation checkpoint to confirm the features available for modeling.
</p>

In [ ]:
# List Features
fc.list_features()

<p style='font-size:16px;font-family:Arial'>
The <code>list_feature_catalogs()</code> function, gives an overview of all curated feature collections (catalogs) 
that have been built using one or more feature processes. 
</p>

In [ ]:
# List feature_catalogs
fs.list_feature_catalogs()

<p style='font-size:16px;font-family:Arial'>
After constructing the curated feature dataset from the Enterprise Feature Store (<b>EFS</b>), 
the next step is to merge these engineered features with selected raw transactional attributes from the original dataset. 
This ensures that the model benefits from both derived and native variables, enriching the learning context.
</p>

In [ ]:
# Join with features with the original table
final_df = tdf.join(
    txn_data.select(['txn_id','type','amount','isFraud','isFlaggedFraud']),
    on='txn_id',
    lprefix="f",
    rprefix="t"
)

In [ ]:
# Renaming txn_id after joining
modeling_df = final_df.assign(
    txn_id = final_df.f_txn_id
)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Data Preparation using widgets</b>

<p style='font-size:16px;font-family:Arial'><b>We'll perform the following steps:</b></p>
<ul style='font-size:16px;font-family:Arial'>
    <li>We will one-hot encode the categorical "type" column.</li>
    <li>We will perform feature scaling using ScaleFit and ScaleTransform on numerical columns.</li>
    <li>We will split the data into training and testing datasets (80:20 split).</li>
</ul>

<p style='font-size:16px;font-family:Arial'>We perform feature scaling during data pre-processing to handle highly varying magnitudes, values, or units. If feature scaling is not done, then a machine learning algorithm tends to weigh greater values higher and consider smaller values as lower ones, regardless of the unit of the values.</p>

<br>
<p style = 'font-size:18px;font-family:Arial'><b>5.1 Drop redundant columns</b></p>
<p style = 'font-size:16px;font-family:Arial'>We don't need nameDest, nameOrigin, isFlaggedFraud, f_txn_id and t_txn_id for model training as they do not impact the outcome. We have txn_id to uniquely identify each transaction.</p>

In [ ]:
# Drop redundant columns
txn_data_df = modeling_df.drop(['nameDest', 'nameOrig', 'isFlaggedFraud', 'f_txn_id', 't_txn_id'], axis = 1)
txn_data_df

<p style = 'font-size:16px;font-family:Arial'>We will copy this dataset to a Vantage table to be used in widgets for Data Preparation</p>


In [ ]:
copy_to_sql(txn_data_df, table_name='transaction_data_new', if_exists='replace')

<br>
<p style = 'font-size:18px;font-family:Arial'><b>5.2 Checking for Outliers using widgets</b></p>
<p style = 'font-size:16px;font-family:Arial'>The OutlierFilterFit() function calculates the lower percentile, upper percentile, count of rows and median for all the "target_columns" provided by the user. These metrics for each column help the function OutlierTransform() detect outliers in data.</p>

<p style = 'font-size:16px;font-family:Arial'>Here we are using teradataml syntax for the function. The same can be achived using the following SQL as well.</p>

<code>SELECT * FROM TD_OutlierFilterFit(
    ON "DEMO_GLM_Fraud"."transaction_data" AS InputTable
    OUT TABLE OutputTable("DEMO_USER"."Outlier_output")
    USING
    TargetColumns('amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig')
) as dt;</code>

<p style = 'font-size:14px;font-family:Arial'><b><i>*Please note that both the versions run in-database and there is no data transfer involved.</i></b></p>

In [ ]:
%load_ext autoreload
%autoreload 2
from teradatamlwidgets.analytic_functions.Ui import *

<p style = 'font-size:16px;font-family:Arial'>As we are showcasing widgets here, we will have to follow some specific steps to execute this function.</p>

<p style='font-size:16px;font-family:Arial'><b>After we execute the below cell it will showcase the widgets output where you will have to enter the below details </b></p>
    <li style='font-size:16px;font-family:Arial'>Select a functions: Please select <b>OutlierFilterFit</b></li>
    
<p style='font-size:16px;font-family:Arial'>After select the function we will see the widget for entering the Inputs we need to provide for the function. Below are the inputs to be provided</p> 
    <li style='font-size:16px;font-family:Arial'><code>InputTable:</code> Select <b>transaction_data_new</b></li>
    <li style='font-size:16px;font-family:Arial'><code>TargetColumns:</code> Please select below columns 1 in each box <b>'amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig'</b></li>
</p>

<p style='font-size:16px;font-family:Arial'>After doing this click on the <b>Execute</b> button and check the output</p>


In [ ]:
inputs = ["transaction_data","transaction_data_new"]
outputs = ["outlierfit_table"]
ui = Ui(
    outputs = outputs,
    inputs = inputs
)

<p style = 'font-size:16px;font-family:Arial'>After OutlierFilterFit we will execute the OutlierFilterTransform function.</p>

<p style='font-size:16px;font-family:Arial'><b>After we execute the below cell it will showcase the widgets output where you will have to enter the below details </b></p>
    <li style='font-size:16px;font-family:Arial'>Select a functions: Please select <b>OutlierFilterTransform</b></li>
    
<p style='font-size:16px;font-family:Arial'>After select the function we will see the widget for entering the Inputs we need to provide for the function. Below are the inputs to be provided</p> 
    <li style='font-size:16px;font-family:Arial'><code>InputTable:</code> Select <b>transaction_data_new</b></li>
    <li style='font-size:16px;font-family:Arial'><code>FitTable:</code> Please select <b> outlierfit_table</b></li>
</p>

<p style='font-size:16px;font-family:Arial'>After doing this click on the <b>Execute</b> button and check the output</p>


In [ ]:
inputs = ['transaction_data_new','outlierfit_table']
outputs = ['outlier_trans_table']
ui = Ui(
    outputs = outputs,
    inputs = inputs,
    function="OutlierFilterTransform",
)

In [ ]:
res = DataFrame('outlier_trans_table')

In [ ]:
print(f"Rows before removing outliers: {txn_data_df.shape[0]}\n\
Rows after removing outliers: {res.shape[0]}\n\
Total outliers: {txn_data_df.shape[0] - res.shape[0]}")

In [ ]:
outliers = td_minus([txn_data_df, res])
outliers

<br>
<p style = 'font-size:18px;font-family:Arial'><b>5.3 One-hot encoding</b></p>
<p style='font-size:16px;font-family:Arial'>
Here, we are one-hot encoding the "type" column. We find one-hot encoding necessary in many cases to represent categorical variables as binary values, enable numerical processing, ensure feature independence, handle non-numeric data, and improve the performance and interpretability of our machine learning models.
</p>

In [ ]:
txn_type_encoder = OneHotEncoder(
    values = ["CASH_IN", "CASH_OUT", "DEBIT", "PAYMENT", "TRANSFER"],
    columns = "type"
)

retain = Retain(
    columns = ['step', 'amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig', 'isFraud']
)

obj = valib.Transform(
    data = txn_data,
    one_hot_encode = txn_type_encoder,
    retain = retain,
    index_columns = 'txn_id'
)
txn_trans = obj.result
txn_trans

In [ ]:
txn_trans = txn_trans.assign(isFraud=txn_trans.isFraud.cast(type_=INTEGER()))
txn_trans                             

<p style = 'font-size:16px;font-family:Arial'>The above output shows that we have transformed the data into a transfromed dataset.</p>

In [ ]:
copy_to_sql(txn_trans, table_name = 'clean_data', if_exists = 'replace')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>6. Create training and testing datasets in Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>We'll create two datasets for training and testing in the ratio of 80:20.</p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(
    data = txn_trans,
    id_column = "txn_id",
    train_size = 0.80,
    test_size = 0.20,
    seed = 25
)

df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

print("Training Set = " + str(df_train.shape[0]) + ". Testing Set = " + str(df_test.shape[0]))

In [ ]:
copy_to_sql(df_train, table_name = 'clean_data_train', if_exists = 'replace')
copy_to_sql(df_test, table_name = 'clean_data_test', if_exists = 'replace')

In [ ]:
df_train

<p style = 'font-size:16px;font-family:Arial'>The above output shows that we have transformed the data into a scaled dataset. Scaling our data makes it easy for our model to learn and understand the problem.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>7. In-Database model training</b>

<p style = 'font-size:16px;font-family:Arial'>ClearScape Analytics provides vertical and horizontal scaling capabilities that make it possible to efficiently train any number of models — from a few to a few million. With a variety of model tarining functions we can build and train high quality ML models, which leverages the analytic datasets. These set of ML components and capabilities help to reduce effort, lower costs, and get ML models into production as quickly as possible</p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will be using In-DB XGBoost and DecisionForest functions to train our models.</p>

<br>
<b style = 'font-size:18px;font-family:Arial'>7.1 XGBoost model training</b>
<p style = 'font-size:16px;font-family:Arial'>The XGBoost() function, also known as eXtreme Gradient Boosting, is an implementation of the gradient boosted decision tree algorithm designed for speed and performance. It has recently been dominating applied machine learning.</p>
<p style = 'font-size:16px;font-family:Arial'>In gradient boosting, each iteration fits a model to the residuals (errors) of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.</p>

<p style = 'font-size:16px;font-family:Arial'>Here we are using teradataml syntax for the function. The same can be achived using the following SQL as well.</p>

<code>SELECT * FROM TD_XGBoost(
	ON "DEMO_USER"."clean_data_train" AS "input"
	PARTITION BY ANY
	USING InputColumns('amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig','CASH_IN_type','CASH_OUT_type','DEBIT_type','PAYMENT_type','TRANSFER_type')
	ResponseColumn('isFraud')
	MaxDepth(7)
	Seed(42)
	ModelType('Classification')
	RegularizationLambda(120.0)
	ShrinkageFactor(0.1)
) as sqlmr</code>

<p style = 'font-size:14px;font-family:Arial'><b><i>*Please note that both the versions run in-database and there is no data transfer involved.</i></b></p>

In [ ]:
cols = df_train.columns
cols.remove('txn_id')
cols.remove('step')
cols.remove('isFraud')

In [ ]:
XGBoost_out = XGBoost(
    data=df_train,
    input_columns=cols,
    response_column = 'isFraud',
    lambda1 = 120.0,
    model_type='Classification',
    seed=42,
    shrinkage_factor=0.1,
    max_depth=7
)

In [ ]:
XGBoost_out.output_data

<p style = 'font-size:16px;font-family:Arial'>The function output is a trained XGBoost model, and we can input it to the XGBoostPredict() function for prediction.</p>

<br>
<p style = 'font-size:18px;font-family:Arial'><b>7.2 Decision Forest model Training</b></p>
<p style = 'font-size:16px;font-family:Arial'>The <a href = 'https://docs.teradata.com/search/all?query=TD_DecisionForest&content-lang=en-US'>DecisionForest</a> model function is an ensemble algorithm used for classification and regression predictive modeling problems. It is an extension of bootstrap aggregation (bagging) of decision trees. Typically, constructing a decision tree involves evaluating the value for each input feature in the data to select a split point. The function reduces the features to a random subset (that can be considered at each split point); the algorithm can force each decision tree in the forest to be very different to improve prediction accuracy. </p>

<p style = 'font-size:16px;font-family:Arial'>This function takes the training data as input, as well as the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li style = 'font-size:16px;font-family:Arial'>InputColumns; list or range of columns used as features (we used an ordinal reference of columns 2:217)</li>
        <li style = 'font-size:16px;font-family:Arial'>ResponseColumn; the dependent or target value (we used “class”, the first column)</li>
        <li style = 'font-size:16px;font-family:Arial'>TreeType; either CLASSIFICATION or REGRESSION</li>
    <li style = 'font-size:16px;font-family:Arial'>Other hyperparameter values detailed in the documentation</li>
        </ul>

In [ ]:
df_train_new = DataFrame('clean_data_train')
df_train_new

In [ ]:
DecisionForest_out = DecisionForest(data = df_train_new, 
                            input_columns = cols, 
                            response_column = 'isFraud', 
                            max_depth = 16, 
                            num_trees = 8, 
                            min_node_size = 1, 
                            mtry = 3, 
                            mtry_seed = 3, 
                            seed = 3, 
                            tree_type = 'CLASSIFICATION')
# Print the result DataFrame.
DecisionForest_out.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>8. In-Database model scoring</b>

<p style = 'font-size:16px;font-family:Arial'>ClearScape Analytics integrates model scoring with business data, both in real time and batch scoring, for effective operationalization and automated monitoring of AI models.</p>
<p style = 'font-size:16px;font-family:Arial'>By deploying these models to conduct live data scoring, Vantage delivers the crucial insights needed to drive business outcomes</p>

<br>
<b style = 'font-size:18px;font-family:Arial'>8.1 XGBoost model scoring</b>
<p style = 'font-size:16px;font-family:Arial'>The XGBoostPredict() function runs the predictive algorithm based on the model generated by XGBoost(). The XGBoost() function, also known as eXtreme Gradient Boosting, performs classification or regression analysis on datasets.</p>


In [ ]:
XGBoostPredict_out = XGBoostPredict(
    newdata=df_test,
    object=XGBoost_out.result,
    model_type='Classification',
    id_column='txn_id',
    object_order_column=['task_index', 'tree_num',
                       'iter', 'tree_order'],
    accumulate='isFraud',
    output_prob=True,
    output_responses=['0', '1']
).result

In [ ]:
XGBoostPredict_out

<p style = 'font-size:16px;font-family:Arial'>The output above shows our prob_1, i.e., the transaction is fraud, and prob_0, i.e., the transaction is not a fraud. We use these probabilities in our prediction column to assign a class label.</p>

<br>
<b style = 'font-size:18px;font-family:Arial'>8.2 Decision Forest model scoring</b>
<p style = 'font-size:16px;font-family:Arial'>The DecisionForestPredict() function uses the model generated by the DecisionForest() function to generate predictions on a response variable for a test set of data. The model can be stored in either a teradataml DataFrame or a DecisionForest object.</p>


In [ ]:
df_test_new = DataFrame('clean_data_test')
df_test_new

In [ ]:
decision_forest_predict_out = TDDecisionForestPredict(object = DecisionForest_out.result,
                                                        newdata = df_test_new,
                                                        id_column = "txn_id",
                                                        detailed = False,
                                                        output_prob = True,
                                                        output_responses = ['0','1'],
                                                        accumulate = 'isFraud')
df_predict = decision_forest_predict_out.result
df_predict

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>9. In-Database model evaluation</b>

<p style = 'font-size:16px;font-family:Arial'>ClearScape Analytics model evaluation report highlights the model performance in the form of certain metrics and compare models based on the metric values.</p>
<p style = 'font-size:16px;font-family:Arial'>The ClassificationEvaluator() function evaluate and emits various metrics of classification model based on its predictions on the data. Apart from accuracy, the secondary output data returns micro, macro, and weighted-averaged metrics of precision, recall, and F1-score values.</p>

In [ ]:
df_predict_char = df = df_predict.assign(isFraud = df_predict.isFraud.cast(type_=VARCHAR(2))
                                        ,prediction_ch = df_predict.prediction.cast(type_=VARCHAR(2)))
df_predict_char

In [ ]:
combined_df = df_test.join(XGBoostPredict_out, on='txn_id', lsuffix='test', rsuffix='pred')
combined_df[combined_df['Prediction']==1]

In [ ]:
out = XGBoostPredict_out.assign(Prediction = XGBoostPredict_out.Prediction.cast(type_ = BYTEINT))
out = out.assign(Prediction = out.Prediction.cast(type_ = VARCHAR(2)))
out = out.assign(isFraud = out.isFraud.cast(type_ = VARCHAR(2)))

In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(
    data = out,
    observation_column = 'isFraud',
    prediction_column = 'Prediction',
    labels = ['0', '1']
)
xgb_eval = ClassificationEvaluator_obj.output_data
xgb_eval.head(10)

In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(
    data = df_predict_char,
    observation_column = 'isFraud',
    prediction_column = 'prediction_ch',
    labels = ['0', '1']
)
df_eval = ClassificationEvaluator_obj.output_data
df_eval.head(10)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>10. Visualize the results (ROC curve and AUC)</b>

<p style = 'font-size:16px;font-family:Arial'>We create the ROC curve, which is a graph between TPR (True Positive Rate) and FPR (False Positive Rate). We use the area under the ROC curve as a metric to evaluate how well our model can distinguish between positive and negative classes. A higher AUC indicates better performance in distinguishing between the positive and negative categories. We generally consider an AUC above 0.75 as decent.</p>

In [ ]:
from teradataml import ROC

xgb_roc_out = ROC(
    probability_column = '"Prob_1"',
    observation_column = "isFraud",
    positive_class = "1",
    data = XGBoostPredict_out,
    num_thresholds=300
)

xgb_roc_out

In [ ]:
from teradataml import ROC

df_roc_out = ROC(
    probability_column = '"prob_1"',
    observation_column = "isFraud",
    positive_class = "1",
    data = df_predict,
    num_thresholds=300
)

df_roc_out

In [ ]:
# Assigning new index column
xgb_roc_out.result = xgb_roc_out.result.assign(row = 1)
# Changing the index label.
xgb_roc_out.result._index_label = ["row"]
xgb_auc = xgb_roc_out.result.get_values()[0][0]

df_roc_out.result = df_roc_out.result.assign(row = 1)
# Changing the index label.
df_roc_out.result._index_label = ["row"]
df_auc = df_roc_out.result.get_values()[0][0]

In [ ]:
from teradataml import subplots
fig, axis = subplots(1, 2)
fig.height, fig.width = 600,1200
plot = xgb_roc_out.output_data.plot(x=xgb_roc_out.output_data.fpr,
    y=[xgb_roc_out.output_data.tpr, xgb_roc_out.output_data.fpr],
    xlabel='False Positive Rate',
    ylabel='True Positive Rate',
    figure=fig,
    ax=axis[0],
    color='carolina blue',
    legend=[f'XGBoost AUC = {round(xgb_auc, 4)}', 'AUC Baseline'],
    legend_style='lower right',
    grid_linestyle='--',
    grid_linewidth=0.5,
    linestyle = ['-', '--'])

plot = df_roc_out.output_data.plot(
    x=df_roc_out.output_data.fpr,
    y=[df_roc_out.output_data.tpr, df_roc_out.output_data.fpr],
    xlabel='False Positive Rate',
    ylabel='True Positive Rate',
    figure=fig,
    ax=axis[1],
    color='carolina blue',
    legend=[f'DecisionForest AUC = {round(df_auc, 4)}', 'AUC Baseline'],
    legend_style='lower right',
    grid_linestyle='--',
    grid_linewidth=0.5,
    linestyle = ['-', '--']
)

plot.show()

<p style = 'font-size:16px;font-family:Arial'>Looking at the above ROC Curve, we can confidently say that our model has performed well on testing data. The AUC value is above 0.75 and resonates with our understanding that the model is performing well.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Conclusion</b></p>

<p style = 'font-size:16px;font-family:Arial'>In this demonstration, we have illustrated a simplified - but complete - overview of how we can implement a typical machine learning workflow completely inside the database using Vantage. This allows us to leverage Vantage's operational scale, power, and stability.</p>

<hr style="height:2px;border:none">
<b style = 'font-size:20px;font-family:Arial'>11. ModelOps for Financial Fraud Detection</b></p>

<p style = 'font-size:16px;font-family:Arial'>We used feature store to store features as well as its processing. We re-used it in model training. The features and processing can be re-used accross multiple machine learning models and use-case , helping to improve data science productivity</p>

<p style = 'font-size:16px;font-family:Arial'>Teradata's traditional approach using Clearscape Analytic functions play a crucial role in this context by automating the complex process of building and deploying machine learning models. Various Clearscape Analytic functions are used for optimal preparation and training of models, delivering high-quality machine learning models in minutes. With the capabilities of ClearScape Analytics ModelOps, Analytics-driven organizations can follow a mature methodology and automated capabilities to solve this gap and make efficient model operationalization at Scale in Vantage.</p>

<p style = 'font-size:16px;font-family:Arial'>ClearScape Analytics ModelOps manages the operationalization of advanced analytics in Teradata Vantage providing Deployment, Governance and Monitoring of your AI/ML models at scale. ModelOps provides an easy-to-use web-based user interface (UI), a command line interface (CLI) and Python/R Software Development Kit (SDK).</p>

<p style = 'font-size:16px;font-family:Arial'>As a part of this End-to_End demo for Financial Fraud Detection, we will implement the ModelOps cycle using Vantage In-Db Vantage functions. Click the below button which will showcase the steps required for the ModelOps cycle as a part of Operatinilizing the model.</p>

<a href="FinFraud_EndtoEnd_ModelOps_GIT_Python_indb_XGB.ipynb" style="display: inline-flex; align-items: center; justify-content: center; background-color: #007373; color: #FFFFFF; font-family: Arial, sans-serif; font-size: 16px; font-weight: bold; text-decoration: none; padding: 12px 24px; border: none; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); cursor: pointer; transition: all 0.3s ease;">
  LAUNCH Notebook for ModelOps
  <img src="https://img.icons8.com/ios-filled/50/ffffff/external-link.png" alt="External Link Icon" style="margin-left: 8px; width: 20px; height: 20px;">
</a>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>12. Cleanup</b>

<div class="alert alert-block alert-warning">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b>The tables created in this demo will be used in the ModelOps notebook which can be invoked on click of the above button. Please uncomment the below lines of code in case you do not want to run the ModelOps notebook and want to delete the tables created for this demo.</p>
</div>    

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
views = ["fraud_week1_view", "fraud_week2_view"]
for view in views:
    try:
        db_drop_view(view_name=view)
    except:
        pass

In [ ]:
tables = ['clean_data', 'clean_data_train', 'clean_data_test']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

In [ ]:
fs.delete()

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../../run_procedure.py "call remove_data('Demo_glm_fraud');"        # Takes 5 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;">

<b style = 'font-size:20px;font-family:Arial'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial'><b>Filters:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Industry:</b> Finance</li>
    <li><b>Functionality:</b> Machine Learning</li>
    <li><b>Use Case:</b> Fraud Detection</li>
</ul>

<p style = 'font-size:18px;font-family:Arial'><b>Related Resources:</b></p>

<ul style = 'font-size:16px;font-family:Arial'>
    <li><a href='https://www.teradata.com/Blogs/Fraud-Busting-AI'>Fraud-Busting-AI</a></li>
    <li><a href='https://www.teradata.com/Industries/Financial-Services'>Financial Services</a></li>
    <li><a href='https://www.teradata.com/Resources/Datasheets/Move-from-Detection-to-Prevention-and-Outsmart-Fraudsters'>Move from Detection to Prevention and Outsmart Tech-Savvy Fraudsters</a></li>
</ul>

<b style = 'font-size:20px;font-family:Arial'>Dataset:</b>

- `txn_id`: transaction id
- `step`: maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (31 days simulation).
- `type`: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
- `amount`: amount of the transaction in local currency
- `nameOrig`: customer who started the transaction
- `oldbalanceOrig`: customer's balance before the transaction
- `newbalanceOrig`: customer's balance after the transaction
- `nameDest`: customer who is the recipient of the transaction
- `oldbalanceDest`: recipient's balance before the transaction
- `newbalanceDest`: recipient's balance after the transaction
- `isFraud`: identifies a fraudulent transaction (1) and non fraudulent (0)
- `isFlaggedFraud`: flags illegal attempts to transfer more than 200,000 in a single transaction

<p style = 'font-size:18px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Uses a dataset and feature discovery methods outlined here: <a href = 'https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook'>https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook</a></li>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>